In [2]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_classification, make_regression
def feedforward_example():
    X, y = make_classification(
        n_samples=1000, 
        n_features=20, 
        n_informative=10, 
        n_classes=2, 
        random_state=42
    )
    
    # Preprocess data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Build and train the model
    model = keras.Sequential([
        layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
        layers.Dropout(0.3),
        layers.Dense(32, activation='relu'),
        layers.Dropout(0.2),
        layers.Dense(16, activation='relu'),
        layers.Dense(1, activation='sigmoid')
    ])
    
    model.compile(
        optimizer='adam', 
        loss='binary_crossentropy', 
        metrics=['accuracy']
    )
    
    # Train the model
    history = model.fit(
        X_train_scaled, y_train, 
        validation_split=0.2, 
        epochs=50, 
        batch_size=32,
        verbose=1
    )
    
    # Evaluate the model
    test_loss, test_accuracy = model.evaluate(X_test_scaled, y_test, verbose=0)
    print(f"\nTest Accuracy: {test_accuracy:.4f}")
    
    return model

# 2. CNN Example: Handwritten Digit Classification
def cnn_mnist_example():
    # Load and preprocess MNIST dataset
    (X_train, y_train), (X_test, y_test) = keras.datasets.mnist.load_data()
    
    # Normalize and reshape
    X_train = X_train.reshape(-1, 28, 28, 1).astype('float32') / 255
    X_test = X_test.reshape(-1, 28, 28, 1).astype('float32') / 255
    y_train = keras.utils.to_categorical(y_train, 10)
    y_test = keras.utils.to_categorical(y_test, 10)
    data_augmentation = keras.Sequential([
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.1),
        layers.RandomZoom(0.1),
    ])
    
    # Build the CNN model
    model = keras.Sequential([
        # Data Augmentation Layer
        data_augmentation,
        
        # Convolutional Layers
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        
        # Flatten and Dense Layers
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(64, activation='relu'),
        layers.Dense(10, activation='softmax')
    ])
    
    # Compile the model
    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    # Train the model
    history = model.fit(
        X_train, y_train, 
        validation_data=(X_test, y_test),
        epochs=10, 
        batch_size=64,
        verbose=1
    )
    test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
    print(f"\nTest Accuracy: {test_accuracy:.4f}")
    
    return model

# 3. RNN Example: Time Series Prediction
def rnn_time_series_example():
    def create_time_series_dataset(n_samples=1000, time_steps=50):
        # Generate sine wave with some noise
        time = np.linspace(0, 10, n_samples)
        series = np.sin(time) + np.random.normal(0, 0.1, n_samples)
        
        # Create sequences
        X, y = [], []
        for i in range(len(series) - time_steps):
            X.append(series[i:i+time_steps])
            y.append(series[i+time_steps])
        
        return np.array(X).reshape(-1, time_steps, 1), np.array(y)
    X, y = create_time_series_dataset()
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Build RNN model
    model = keras.Sequential([
        layers.LSTM(50, activation='relu', input_shape=(X_train.shape[1], 1)),
        layers.Dense(25, activation='relu'),
        layers.Dense(1)
    ])
    
    # Compile model
    model.compile(
        optimizer='adam', 
        loss='mse',
        metrics=['mae']
    )
    
    # Train model
    history = model.fit(
        X_train, y_train, 
        validation_data=(X_test, y_test),
        epochs=50, 
        batch_size=32,
        verbose=1
    )
    test_loss, test_mae = model.evaluate(X_test, y_test, verbose=0)
    print(f"\nTest Mean Absolute Error: {test_mae:.4f}")
    
    return model
def run_neural_network_examples():
    print("1. Feedforward Neural Network Example:")
    feedforward_model = feedforward_example()
    print("\n2. Convolutional Neural Network (MNIST) Example:")
    cnn_model = cnn_mnist_example()   
    print("\n3. Recurrent Neural Network (Time Series) Example:")
    rnn_model = rnn_time_series_example()